In [ ]:
import os, re, time, textwrap, subprocess, sys
from pathlib import Path

def sh(cmd, check=True, env=None, cwd=None):
    print(f"$ {cmd}")
    p = subprocess.run(cmd, shell=True, env={**os.environ, **(env or {})} if env else None,
                       cwd=cwd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(p.stdout)
    if check and p.returncode!=0: raise RuntimeError(p.stdout)
    return p.stdout

In [ ]:
WORK=Path("/content/mle_colab_demo"); WORK.mkdir(parents=True, exist_ok=True)
PROJ=WORK/"proj"; PROJ.mkdir(exist_ok=True)
DATA=WORK/"data.csv"; MODEL=WORK/"model.joblib"; PREDS=WORK/"preds.csv"
SAFE=WORK/"train_safe.py"; RAW=WORK/"agent_train_raw.py"; FINAL=WORK/"train.py"
MODEL_NAME=os.environ.get("OLLAMA_MODEL","llama3.2:1b")

sh("pip -q install --upgrade pip")
sh("pip -q install mle-agent==0.4.* scikit-learn pandas numpy joblib")

sh("curl -fsSL https://ollama.com/install.sh | sh")
sv = subprocess.Popen("ollama serve", shell=True)
time.sleep(4); sh(f"ollama pull {MODEL_NAME}")

In [ ]:
import numpy as np, pandas as pd
np.random.seed(0)
n=500; X=np.random.rand(n,4); y=(X@np.array([0.4,-0.2,0.1,0.5])+0.15*np.random.randn(n)>0.55).astype(int)
pd.DataFrame(np.c_[X,y], columns=["f1","f2","f3","f4","target"]).to_csv(DATA, index=False)

env = {"OPENAI_API_KEY":"", "ANTHROPIC_API_KEY":"", "GEMINI_API_KEY":"",
       "OLLAMA_HOST":"http://127.0.0.1:11434", "MLE_LLM_ENGINE":"ollama","MLE_MODEL":MODEL_NAME}
prompt=f"""Return ONE fenced python code block only.
Write train.py that reads {DATA}; 80/20 split (random_state=42, stratify);
Pipeline: SimpleImputer + StandardScaler + LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42);
Print ROC-AUC & F1; print sorted coefficient magnitudes; save model to {MODEL} and preds to {PREDS};
Use only sklearn, pandas, numpy, joblib; no extra text."""
def extract(txt:str)->str|None:
    txt=re.sub(r"\x1B\[[0-?]*[ -/]*[@-~]", "", txt)
    m=re.search(r"```(?:python)?\s*([\s\S]*?)```", txt, re.I)
    if m: return m.group(1).strip()
    if txt.strip().lower().startswith("python"): return txt.strip()[6:].strip()
    m=re.search(r"(?:^|\n)(from\s+[^\n]+|import\s+[^\n]+)([\s\S]*)", txt);
    return (m.group(1)+m.group(2)).strip() if m else None

out = sh(f'printf %s "{prompt}" | mle chat', check=False, cwd=str(PROJ), env=env)
code = extract(out) or sh(f'printf %s "{prompt}" | ollama run {MODEL_NAME}', check=False, env=env)
code = extract(code) if code and not isinstance(code, str) else (code or "")
(Path(RAW)).write_text(code or "", encoding="utf-8")

In [ ]:
def sanitize(src:str)->str:
    if not src: return ""
    s = src
    s = re.sub(r"\r","",s)
    s = re.sub(r"^python\b","",s.strip(), flags=re.I).strip()
    fixes = {
        r"from\s+sklearn\.pipeline\s+import\s+SimpleImputer": "from sklearn.impute import SimpleImputer",
        r"from\s+sklearn\.preprocessing\s+import\s+SimpleImputer": "from sklearn.impute import SimpleImputer",
        r"from\s+sklearn\.pipeline\s+import\s+StandardScaler": "from sklearn.preprocessing import StandardScaler",
        r"from\s+sklearn\.preprocessing\s+import\s+ColumnTransformer": "from sklearn.compose import ColumnTransformer",
        r"from\s+sklearn\.pipeline\s+import\s+ColumnTransformer": "from sklearn.compose import ColumnTransformer",
    }
    for pat,rep in fixes.items(): s = re.sub(pat, rep, s)
    if "SimpleImputer" in s and "from sklearn.impute import SimpleImputer" not in s:
        s = "from sklearn.impute import SimpleImputer\n"+s
    if "StandardScaler" in s and "from sklearn.preprocessing import StandardScaler" not in s:
        s = "from sklearn.preprocessing import StandardScaler\n"+s
    if "ColumnTransformer" in s and "from sklearn.compose import ColumnTransformer" not in s:
        s = "from sklearn.compose import ColumnTransformer\n"+s
    if "train_test_split" in s and "from sklearn.model_selection import train_test_split" not in s:
        s = "from sklearn.model_selection import train_test_split\n"+s
    if "joblib" in s and "import joblib" not in s: s = "import joblib\n"+s
    return s

san = sanitize(code)

safe = textwrap.dedent(f"""
import pandas as pd, numpy as np, joblib
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.compose import ColumnTransformer

DATA=Path("{DATA}"); MODEL=Path("{MODEL}"); PREDS=Path("{PREDS}")
df=pd.read_csv(DATA); X=df.drop(columns=["target"]); y=df["target"].astype(int)
num=X.columns.tolist()
pre=ColumnTransformer([("num",Pipeline([("imp",SimpleImputer()),("sc",StandardScaler())]),num)])
clf=LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
pipe=Pipeline([("pre",pre),("clf",clf)])
Xtr,Xte,ytr,yte=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
pipe.fit(Xtr,ytr)
proba=pipe.predict_proba(Xte)[:,1]; pred=(proba>=0.5).astype(int)
print("ROC-AUC:",round(roc_auc_score(yte,proba),4)); print("F1:",round(f1_score(yte,pred),4))
import pandas as pd
coef=pd.Series(pipe.named_steps["clf"].coef_.ravel(), index=num).abs().sort_values(ascending=False)
print("Top coefficients by |magnitude|:\\n", coef.to_string())
joblib.dump(pipe,MODEL)
pd.DataFrame({{"y_true":yte.reset_index(drop=True),"y_prob":proba,"y_pred":pred}}).to_csv(PREDS,index=False)
print("Saved:",MODEL,PREDS)
""").strip()

In [5]:
chosen = san if ("import " in san and "sklearn" in san and "read_csv" in san) else safe
Path(SAFE).write_text(safe, encoding="utf-8")
Path(FINAL).write_text(chosen, encoding="utf-8")
print("\n=== Using train.py (first 800 chars) ===\n", chosen[:800], "\n...")

sh(f"python {FINAL}")
print("\nArtifacts:", [str(p) for p in WORK.glob('*')])
print("✅ Done — outputs in", WORK)

$ pip -q install --upgrade pip

$ pip -q install mle-agent==0.4.* scikit-learn pandas numpy joblib

$ curl -fsSL https://ollama.com/install.sh | sh
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
#=#=#                                                                         
##O#-#                                                                        
##O=#  #                                                                      
#=#=-#  #                                                                     
-#O#- #   #                                                                   
-=#=#   #   #                                                                 

                                                                           0.0%
                                                                           0.0%
                                                                           0.1%
             